In [1]:
import os
import sys
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp
from vocab import Vocabulary
from env import ConceptData
from create_data import addFile
import tensorflow as tf
import random
import numpy as np
import csv
import re
import os
import create_data
from xml.dom import minidom
import xml.etree.ElementTree as ET
from vocab import Vocabulary
from env import ConceptData
from create_data import addFile
import agents

In [33]:
batch_size = 32
hidden_size = 44
num_distractors = 7
#ist an sich vorgegeben und 597 oder so
num_categories = 58
vocab_size = 99
embed_dim = 50
max_len = 5
num_epochs = 12
sender_entropy_coeff = 0.015 #wie bei Ossenkopf
receiver_entropy_coeff = 0.0 # wie bei Ossenkopf

In [32]:


#im folgenden tun wir so als wäre vocab unsere daten
#vocab ist ninput für Receiver
vocab = tf.random.categorical(tf.math.log([[0.5, 0.5]]), 
                              12992)

vocab = tf.reshape(vocab, [7,32,58])

#Input für sender
speakerInput = vocab[:][:][0]


input_concepts:
[<DOM Element: concept at 0x202fc0eb670>, <DOM Element: concept at 0x202fb311a60>, <DOM Element: concept at 0x202fbfab9d0>, <DOM Element: concept at 0x202fc1448b0>, <DOM Element: concept at 0x202fbf9b670>, <DOM Element: concept at 0x202fc0eb8b0>, <DOM Element: concept at 0x202fb87b040>, <DOM Element: concept at 0x202fc08ce50>, <DOM Element: concept at 0x202fbf865e0>, <DOM Element: concept at 0x202fc0eb4c0>]
target_concept:
<DOM Element: concept at 0x202fc08ce50>


In [4]:
#weil sender noch nicht tut, nemen wir das als dummy für message die eig von sender outgeputtet wird
message = tf.constant([[89, 37, 86, 72,  0],
        [ 9, 85, 11, 20,  0],
        [25, 16, 20, 12,  0],
        [97,  0, 26, 89,  0],
        [45, 86, 34, 43,  0],
        [49, 91, 59, 56,  0],
        [80, 63, 70, 53,  0],
        [96, 57, 98, 54,  0],
        [41, 10, 92, 83,  0],
        [71, 45, 70, 51,  0],
        [89, 59, 45, 52,  0],
        [62, 33, 94, 11,  0],
        [55, 91, 16, 74,  0],
        [17, 77, 15, 78,  0],
        [93, 61, 12, 46,  0],
        [32, 38, 93, 46,  0],
        [73, 86, 78, 85,  0],
        [63, 74, 85, 92,  0],
        [52,  6, 41, 51,  0],
        [ 6, 60, 61, 37,  0],
        [61, 44, 32,  8,  0],
        [ 8, 67, 83, 64,  0],
        [ 4, 74, 57, 96,  0],
        [91, 75, 74, 50,  0],
        [71, 24,  6, 14,  0],
        [26, 79, 59, 43,  0],
        [66, 54, 25,  2,  0],
        [28,  7, 74, 92,  0],
        [12, 59, 44, 93,  0],
        [ 3, 38, 21, 10,  0],
        [14, 35, 80,  2,  0],
        [21, 11, 10, 64,  0]])

In [5]:
#Initialisiere die Agents
speakerEncoder = agents.Encoder(units = 44, categories_dim = 58)
speakerDecoder = agents.Sender_LSTM(vocab_size, embed_dim, hidden_size, max_len )
receiverEncoder = agents.Receiver(num_distractors)
receiverDecoder = agents.Receiver_LSTM(receiverEncoder, vocab_size, embed_dim, hidden_size)
guesser = agents.AuxillaryNetwork(hidden_size)

In [7]:
#prev_hidden kommt von sender; müssen wir hier faken
prev_hidden = tf.random.normal([32,44])

In [8]:
guesser_output = guesser(prev_hidden)

In [19]:
#the losses
#loss um prediction mit Realität zu vergleichen
#für was brauchen wir die underscore variablen?

def loss(_sender_input, _message, _receiver_input, receiver_output, labels):
    """
    receiver_output ist was von receiver_sampling zurückgegeben wird
    LABELS PRINTEN IN OSSSENKOPF NOTEBOOK
    """
    acc = (labels.t() == receiver_output).float() - (labels.t() != receiver_output).float()
    return -acc, {'acc': (acc.mean().item()+1)/2}

#loss um prediction mit Realität zu vergleichen
def auxiliary_loss(receiver_thoughts, 
                  # _message, _receiver_input, 
                   guesser_output, 
                   #_labels,
                   weight=0.2):
    mae = tf.keras.losses.MeanAbsoluteError(reduction = 'none')
    loss = mae(receiver_thoughts, guesser_output,)
    loss *= weight
    return loss, {}

In [36]:
optim_guesser = tf.keras.optimizers.Adam(learning_rate = 1e-2)
optim_receiver = tf.keras.optimizers.Adam(learning_rate = 1e-2)
optim_sender = tf.keras.optimizers.Adam(learning_rate = 1e-3)

In [45]:
for i in range(num_epochs):
    # 1. data holen, labels?
    ####Fehlt hier noch
    
    
    ##2. durch agents laufen lassen
    output_sender = speakerEncoder(speakerInput)
    message, log_prob_s, entropy_s , prev_hidden = speakerDecoder(output_sender)
    sample, log_prob_r, entropy_r, last_hidden = receiverDecoder(message, batch_size, max_len,vocab)
    guesser_output = guesser(prev_hidden)
        
    
    # 3. Loss berechnen
    auxLoss = auxiliary_loss(last_hidden, guesser_output)
    # labels fehlen noch
    loss = loss(sample, labels)
    ##policy_length_loss und das baseline Zeug miteinbringen?
    weighted_entropy = entropy_r * receiver_entropy_coeff + sender_entropy_coeff # *entropy_r
    log_prob = log_prob_r # + log_prob_s
    loss = tf.math.reduce_mean(loss* log_prob) - weighted_entropy
    
    #optimization
    
    with tf.GradientTape() as tape:
        # muss der eine Receiver im Konstruktor vom anderem erstellt werden damit das funktioniert?
        receiver_gradients = tape.gradient(loss, receiverDecoder.trainable_variables)
        print(receiver_gradients)
    optim_receiver.apply_gradients(zip(receiver_gradients, receiverDecoder.trainable_variables))
    
    with tf.GradientTape() as tape:
        guesser_gradients = tape.gradient(auxLoss, guesser.trainable_variables)
        print(guesser_gradients)
    optim_guesser.apply_gradients(zip(guesser_gradients, guesser.trainable_variables))
    
    with tf.GradientTape() as tape:
        senderLSTM_gradients = tape.gradient(loss, speakerDecoder.trainable)
        senderEncoder_gradients = guesser_gradients + senderLSTM_gradients
    optim_sender.apply_gradients(zip(senderLSTM_gradients, speakerDecoder.trainable_variables))
    optim_sender.apply_gradients(zip(senderEncoder_gradients, senderEncoder.trainable_variables))
        
    
    
    
    #testing <-> trainifehlt noch
    